In [1]:
from Data.dataloader import create_data_loaders

# create dataloader randomly
train_loader, val_loader, test_loader = create_data_loaders(
    lmdb_path='./pdbbind.lmdb', 
    batch_size=4,
    train_ratio=0.7,
    val_ratio=0.2,
    test_ratio=0.1,
    seed=42
)

In [2]:
# transfer train_loader to list
train_list = list(train_loader)

for protein_batch, ligand_batch, kd_values, _ in train_loader:
    break

In [3]:
protein_batch

DataBatch(x=[2242, 1284], edge_index=[2, 22456], pos=[2242, 3], batch=[2242], ptr=[5])

In [4]:
ligand_batch

DataBatch(x=[158, 9], edge_index=[2, 324], edge_attr=[324, 3], pos=[158, 3], batch=[158], ptr=[5])

In [5]:
kd_values

tensor([4.4200, 4.4000, 4.3100, 4.3300])

In [6]:
from Model.MoleculeEncoder.mpnn import MPNN
import torch.nn as nn
from Model.MoleculeEmbedding.embedding import MoleculeEmbedding
x_embedded, edge_attr_embedded = MoleculeEmbedding(ligand_batch.x,ligand_batch.edge_attr)
ligand_batch.x = x_embedded
ligand_batch.edge_attr = edge_attr_embedded
ligand_encoder = MPNN(input_dim=256, 
                        edge_dim=64, 
                        hidden_dim=128,
                        output_dim=128,
                        num_layers=3,
                        activation=nn.ReLU(),
                        dropout=0.1)

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
print(type(ligand_batch.x), ligand_batch.x.shape)
print(type(ligand_batch.edge_attr), ligand_batch.edge_attr.shape)

<class 'torch.Tensor'> torch.Size([158, 9])
<class 'torch.Tensor'> torch.Size([324, 3])


In [ ]:
ligand_encoder(ligand_batch)

AttributeError: 'MoleculeEmbedding' object has no attribute 'x'

In [ ]:
print(ligand_batch.x.dtype)
print(ligand_encoder.input_proj.weight.dtype)

torch.int64


NameError: name 'ligand_encoder' is not defined